In [17]:
import datetime
import os
import io
import re
import json
import numpy as np
import pandas as pd
import scipy.stats
import boto3

In [90]:
def CONFIG():
    """"""
    _pid = 'analysis-covid19'
    _now = datetime.datetime.now()
    _cwd = os.getcwd()
    _data = os.path.join(_cwd,'data')
    
    _cfg = {
        'data': _data
        ,'csse': os.path.join(_data,'csse-dly','{DT}.csv')
        ,'csse-us': os.path.join(_data,'csse-dly-us','{DT}.csv')
        ,'csse-ts': {
            'base': os.path.join(_data,'csse-ts','time_series_covid19_{STAT}_{RGN}.csv')
            ,'options': {
                'stat': ['confirmed','deaths','recovered']
                ,'rgn': ['global','US']
            }
            ,'errata': 'errata.csv'
        }
        ,'who-ts': os.path.join(_data,'who-ts','who_covid_19_sit_rep_time_series.csv')
    }
    
    return _cfg

CFG = CONFIG()

In [91]:
def CSSE_DLY(base,start=datetime.date(2020,4,12),end=datetime.date(2020,4,30)):
    """"""
    _file = '%m-%d-%Y'
    _fmt = '%Y%m%d'
    df = None
    gaps = []
    
    for i in range((end-start).days+1):
        dt = (start+datetime.timedelta(days=i))
        if df is not None:
            try:
                df.append(pd.read_csv(base.format(DT=dt.strftime(_file)),header=0).assign(data_dt=dt.strftime(_fmt)))
            except Exception as e:
                gaps.append((dt,str(e)))
        else:
            try:
                df = pd.read_csv(base.format(DT=dt.strftime(_file)),header=0).assign(data_dt=dt.strftime(_fmt))
            except Exception as e:
                gaps.append((dt,str(e)))
    
    return df # gaps

In [ ]:
def CSSE_TS(path):
    """"""
    df = None
    
    return df

In [92]:
def CLEANER(df):
    """"""
    def _COLS(x):
        """"""
        if not x[0].isalnum():
            x=x[1:]
        if not x[-1].isalnum():
            x=x[:-1]
        return x.upper()
    # cols
    df.columns = map(lambda x: _COLS(x),df.columns)
    
    return df

In [93]:
if __name__=='__main__':
# def MAIN():
#     """"""
    data = {
        'csse': None,
        'csse-us': None,
        'csse-ts': None,
        'who-ts': None
    }
    ERRS = []
    CFG = CONFIG()
    
    data['csse'] = CLEANER(CSSE_DLY(CFG['csse']))
    data['csse-us']  = CLEANER(CSSE_DLY(CFG['csse-us']))
    data['csse-ts'] = CLEANER(pd.read_csv(CFG['csse-ts']['base'].format(STAT='confirmed',RGN='US'),header=0)) # ['confirmed-us']
    data['who-ts'] = CLEANER(pd.read_csv(CFG['who-ts'],header=0))
    
#     return data